# Cleaning and Analyzing Employee Exit Surveys

Feedback from employee exit surveys can provide powerful insight into a company’s culture. It doesn't matter how excellent a company is, people are eventually going to leave. Exit surveys allows leaving employees to share their unique opinions. This can help companies in mitigating the many costs of losing other employees in the future.

![](https://images.squarespace-cdn.com/content/v1/5dd58dee80e8fe0621b369a3/1630337526747-0Y46T61FP1AFLQ9X9A77/Employee+Exiting+An+Organization?format=1000w)

Image source: [Skywalk Group](https://skywalkgroup.com/blog/post/the-importance-of-exit-interviews)

## Project and Data Overview
In this Project, we'll work with exit surveys from employees of the Department of Education, Training and Employment (DETE) and the Technical and Further Education (TAFE) institute in Queensland, Australia. 

The DETE exit survey data can be found [here](https://data.gov.au/dataset/ds-qld-fe96ff30-d157-4a81-851d-215f2a0fe26d/details?q=exit%20survey). However, the original TAFE exit survey data is no longer available. Some modifications have been made to the original datasets to make them easier to work with, especially changing the encoding from cp1252 to UTF-8.

## Business Problem
We will play the role of data analyst and pretend our stakeholders want to know the following:
- Is some dissatisfaction causing newer and older employees to resign from the institute?
- If a dissatisfaction is present, how does vary within the different age groups at the instititute? 
- Are females more likely to resign than males?

The stakeholders want us to combine the results from both surveys and answer these questions. Although both surveys used the same template, one of them had customized answers. 

## Data Dictionary
A data dictionary wasn't provided with the dataset. In a job setting, we'd **make sure** to meet with a manager to confirm the definitions of the data. For this project, we'll use our general knowledge to define the columns.

From *`dete_survey.csv`*, we will focus on the following columns:
- `ID`: An id used to identify the participant of the survey.
- `SeparationType`: The reason why the person's employment ended.
- `Cease Date`: The year or month the person's employment ended.
- `DETE Start Date`: The year the person began employment with the DETE.

From *`tafe_survey.csv`*, we will focus on the following columns:
- `Record ID`: An id used to identify the participant of the survey.
- `Reason for ceasing employment`: The reason why the person's employment ended.
- `LengthofServiceOverall`. Overall Length of Service at Institute (in years): The length of the person's employment (in years).

## Useful Libraries

In [1]:
from datetime import datetime
from tabulate import tabulate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

## A. Exploring the DETE Dataset

In [2]:
#read the dete dataset
dete_survey = pd.read_csv('./dete_survey.csv')

# preview dataset info
dete_survey.info()
dete_survey.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822 entries, 0 to 821
Data columns (total 56 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   ID                                   822 non-null    int64 
 1   SeparationType                       822 non-null    object
 2   Cease Date                           822 non-null    object
 3   DETE Start Date                      822 non-null    object
 4   Role Start Date                      822 non-null    object
 5   Position                             817 non-null    object
 6   Classification                       455 non-null    object
 7   Region                               822 non-null    object
 8   Business Unit                        126 non-null    object
 9   Employment Status                    817 non-null    object
 10  Career move to public sector         822 non-null    bool  
 11  Career move to private sector        822 non-

,ID,SeparationType,Cease Date,DETE Start Date,Role Start Date,Position,Classification,Region,Business Unit,Employment Status,...,Kept informed,Wellness programs,Health & Safety,Gender,Age,Aboriginal,Torres Strait,South Sea,Disability,NESB
0,1,Ill Health Retirement,08/2012,1984,2004,Public Servant,A01-A04,Central Office,Corporate Strategy and Peformance,Permanent Full-time,...,N,N,N,Male,56-60,NaN,NaN,NaN,NaN,Yes
1,2,Voluntary Early Retirement (VER),08/2012,Not Stated,Not Stated,Public Servant,AO5-AO7,Central Office,Corporate Strategy and Peformance,Permanent Full-time,...,N,N,N,Male,56-60,NaN,NaN,NaN,NaN,NaN
2,3,Voluntary Early Retirement (VER),05/2012,2011,2011,Schools Officer,NaN,Central Office,Education Queensland,Permanent Full-time,...,N,N,N,Male,61 or older,NaN,NaN,NaN,NaN,NaN
3,4,Resignation-Other reasons,05/2012,2005,2006,Teacher,Primary,Central Queensland,NaN,Permanent Full-time,...,A,N,A,Female,36-40,NaN,NaN,NaN,NaN,NaN
4,5,Age Retirement,05/2012,1970,1989,Head of Curriculum/Head of Special Education,NaN,South East,NaN,Permanent Full-time,...,N,A,M,Female,61 or older,NaN,NaN,NaN,NaN,NaN


### Initial Notes
- The dataset comprises 822 rows and 56 columns
- The column names do not conform to the recommended python snake case convention. Infact, the naming convention here appears inconsistent.
- 32 of the 56 columns in the dataset contain missing data. Columns like `Classification`, `Business Unit`, `Aboriginal`, `Torres Strait`, `South Sea`, `Disability` and `NESB` have over 50% missing data. 
- 18 of the 56 columns are stored as boolean data types. Only the `ID` column is stored as an integer type. Other columns are stored as object/string data.
- Time data (`Cease Date`, `DETE Start Date` and `Role Start Date`) are stored as object/string data instead of integer or datetime data.

In [3]:
dete_survey.describe(include='all')

,ID,SeparationType,Cease Date,DETE Start Date,Role Start Date,Position,Classification,Region,Business Unit,Employment Status,...,Kept informed,Wellness programs,Health & Safety,Gender,Age,Aboriginal,Torres Strait,South Sea,Disability,NESB
count,822.000000,822,822,822,822,817,455,822,126,817,...,813,766,793,798,811,16,3,7,23,32
unique,NaN,9,25,51,46,15,8,9,14,5,...,6,6,6,2,10,1,1,1,1,1
top,NaN,Age Retirement,2012,Not Stated,Not Stated,Teacher,Primary,Metropolitan,Education Queensland,Permanent Full-time,...,A,A,A,Female,61 or older,Yes,Yes,Yes,Yes,Yes
freq,NaN,285,344,73,98,324,161,135,54,434,...,401,253,386,573,222,16,3,7,23,32
mean,411.693431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,237.705820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,206.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,411.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,616.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Additional Notes

- The most frequent reason for employee exit from DETE is *age retirement*.
- *'61 or older'* is the most common age group for respondents. This may further support age retirement as the most common reason for exit.
- Most of the entries for `Start Date` and `Role Start Date` were recorded as *'Not Stated'*. There could be a chance that this information wasn't available to respondents at the time of completing the survey.
- The last five columns, `Aboriginal`, `Torres Strait`, `South Sea`, `Disability` and `NESB` have only one unique value which is **'Yes'**. This might explain why they have the highest proportion of null values. Null entries in these columns might have stood for **'No'** at the time the survey was administered.
- The most common entry from the Professional Development column to the Health & Safety column is 'A'. This seems quite unusual as 'A' doesn't seem to represent anything. We will explore these columns further.

We will define a function `count_values()` which computes the counts of all the unique values in a series. Next, we will apply the function to an extracted dataframe using the `.apply()` method:

In [4]:
def count_values(series):
    return series.value_counts()

# extract the columns from Professional Development to Health and Safety.
flagged_columns = dete_survey.iloc[:, 28:49]

# use the apply method to compute the count of unique values in the flagged columns
flagged_columns.apply(count_values)

,Professional Development,Opportunities for promotion,Staff morale,Workplace issue,Physical environment,Worklife balance,Stress and pressure support,Performance of supervisor,Peer support,Initiative,...,Coach,Career Aspirations,Feedback,Further PD,Communication,My say,Information,Kept informed,Wellness programs,Health & Safety
A,413,242,335,357,467,359,342,349,401,396,...,345,246,348,293,399,400,436,401,253,386
D,60,83,112,77,61,107,95,77,37,34,...,65,108,78,77,76,52,45,60,105,50
M,15,24,13,14,15,12,14,12,11,13,...,22,17,15,13,8,10,11,10,33,28
N,103,230,158,160,99,116,168,120,95,95,...,141,183,138,179,129,116,120,130,225,153
SA,184,100,121,115,148,162,124,179,243,244,...,157,130,156,149,144,177,165,162,78,141
SD,33,56,77,65,27,59,67,76,25,31,...,37,62,57,57,58,57,39,50,72,35


From Professional Development to Health & Safety, there are 6 unique values: `A`, `D`, `M`,`N`, `SA`, `SD`.

These may be aliases for the infamous survey options: **Strongly Agree (SA), Moderately agree (M), Agree (A), Neutral (N), Disagree (D) and Strongly Disagree (SD)**. Of these options, Agree (A) seems to be the most commonly selected option.

## B. Exploring the TAFE Dataset

In [5]:
tafe_survey = pd.read_csv('./tafe_survey.csv')

# preview dataset info
tafe_survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 72 columns):
 #   Column                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                         --------------  -----  
 0   Record ID                                                                                                                                                      702 non-null    float64
 1   Institute                                                                                                                                                      702 non-null    object 
 2   WorkArea                                                                                                                                  